# CMRM Assignment No. 2

In [ ]:
import os
import numpy as np
import soundfile as sf
import librosa
import sklearn # pip install sklearn
import IPython.display as ipd
import matplotlib.pyplot as plt
from tqdm import tqdm # pip install tqdm
from nmf import nmf

## Question 1

In [ ]:
# Define directories and lists
output_dir = 'audio/results'
source_dir = 'audio/sources'
target_dir = 'audio/targets'

sources = ["Bees_Buzzing.mp3", "Wind_Blowing.mp3", "Chainsaw_Sawing.mp3"]
targets = ["Jingle_Bells_Boogie.wav", "Have_Yourself.wav", "Blue_Christmas.wav", "White_Christmas.wav"]

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# For initial testing we're using these sources
source = sources[0]  # "Bees_Buzzing.mp3"
target = targets[0]  # "Jingle_Bells_Boogie.wav"



In [ ]:
# Load signals
Fs = 22050

source_wave, _ = librosa.load(os.path.join(source_dir, source).replace("\\", "/"), sr=Fs)
target_wave, _ = librosa.load(os.path.join(target_dir, target).replace("\\", "/"), sr=Fs)

In [ ]:
N_length = 4096
H_size = 1024

# STFT computation
X_source = librosa.stft(source_wave, n_fft=N_length, hop_length=H_size)
X_target = librosa.stft(target_wave, n_fft=N_length, hop_length=H_size)

# Source
Y_source = np.abs(X_source)

# Plot
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Magnitude of Y_source (dB)")
librosa.display.specshow(librosa.amplitude_to_db(Y_source, ref=np.max), sr=Fs, hop_length=H_size, x_axis="time", y_axis="log")
plt.colorbar(format="%.2f dB")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")

# Target
Y_target = np.abs(X_target)

# Plot
plt.subplot(1, 2, 2)
plt.title("Magnitude of Y_target (dB)")
librosa.display.specshow(librosa.amplitude_to_db(Y_target, ref=np.max), sr=Fs, hop_length=H_size, x_axis="time", y_axis="log")
plt.colorbar(format="%.2f dB")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.tight_layout()
plt.show()

#Computing time and frequency resolutions
time_res = H_size / Fs  #Time resolution in seconds
freq_res = Fs / N_length  #Frequency resolution in Hz
print(f"Time Resolution: {time_res:.4f} seconds") #Printing time resolution
print(f"Frequency Resolution: {freq_res:.2f} Hz") #Printing frequqnecy resolution

## Question 2

In [ ]:
# Initialize activations randomly
eps = 1e-8 

W0 = Y_source / (np.sum(Y_source, axis=0, keepdims=True) + eps)

# Normalize the source STFT for Xs
Xs = X_source / (np.sum(Y_source, axis=0, keepdims=True) + eps)

H0 = np.random.rand(X_source.shape[1], X_target.shape[1])

# Initialize templates according to source frames
# NMF
print("Performing NMF...")
W, H = nmf(
    Y_target,
    num_iter=50,
    init_W=W0,
    init_H=H0,
    fix_W=True,
    cont_polyphony=10,
    cont_length=7,
    cont_grid=5,
    cont_sparsen=(1, 7)
)

Y_approx = np.dot(W, H)

# Frobenius error 
error = Y_target - Y_approx
frobenius_norm = np.linalg.norm(error, ord='fro') # Compute the Frobenius norm of the error
print(f"Frobenius norm of the error: {frobenius_norm}")




In [ ]:
# Plot H0 and H
# H0 
plt.figure(figsize=(8, 4))
plt.title("Initial Activation Matrix (H0)")
plt.imshow(H0, aspect="auto", origin="lower", cmap="viridis")
plt.colorbar(label="Value")
plt.xlabel("Time Frames")
plt.ylabel("Components")
plt.tight_layout()
plt.show()

# H
plt.figure(figsize=(8, 4))
plt.title("Learned Activation Matrix (H)")
plt.imshow(H, aspect="auto", origin="lower", cmap="viridis")
plt.colorbar(label="Value")
plt.xlabel("Time Frames")
plt.ylabel("Components")
plt.tight_layout()
plt.show()

#To make components more visible we compress H using a low value of gamma
gamma = 0.1
H_compressed = np.power(H,gamma)

# H compressed plot
plt.figure(figsize=(8, 4))
plt.title("Compressed Activation Matrix (H)")
plt.imshow(H_compressed, aspect="auto", origin="lower", cmap="viridis")
plt.colorbar(label="Value")
plt.xlabel("Time Frames")
plt.ylabel("Components")
plt.tight_layout()
plt.show()

In [ ]:
def visualize_nmf(V, W, H, fs, time_res, gamma=2):
    """
    Visualize the matrices W, H, and compare reconstruction V ~ WH to V.

    Args:
        V: Target magnitude STFT (2D numpy array)
        W: Template matrix (2D numpy array)
        H: Activation matrix (2D numpy array)
        fs: Sampling frequency (float)
        time_res: Time resolution (float)
        gamma: Compression factor for visualization (default is 2)

    Returns:
        None
    """
    #Applying compression for visualization
    V_compressed = np.power(V, gamma) #Compress target Magnitude STFT 
    WH_compressed = np.power(np.dot(W, H), gamma) #Compress reconstructed magnitude STFT
    W_compressed = np.power(W, gamma) #Compress template Matrix
    H_compressed = np.power(H, gamma) #Compress activation Matrix

    #Frequency axis limited to 2000 Hz
    freq_axis = np.linspace(0, 2000, W.shape[0]) #Mapping rows of W
    time_axis_V = np.arange(V.shape[1]) * time_res #Mapping columns of V to time axis
    time_axis_H = np.arange(H.shape[1]) * time_res #Mappinf columns of H to time axis

    #Plotting W (template matrix)
    plt.figure(figsize=(15, 10))
    plt.subplot(2, 2, 1)  #TOPLEFT
    plt.imshow(W_compressed, aspect='auto', origin='lower', cmap='viridis', extent=[0, W.shape[1], freq_axis[0], freq_axis[-1]])
    plt.colorbar(label="Compressed Template Strength")
    plt.title("Template Matrix (W)")
    plt.xlabel("Components")
    plt.ylabel("Frequency [Hz]")

    #Plotting H (activation matrix)
    plt.subplot(2, 2, 2) #TOPRIGHT
    plt.imshow(H_compressed, aspect='auto', origin='lower', cmap='viridis', extent=[time_axis_H[0], time_axis_H[-1], 0, H.shape[0]])
    plt.colorbar(label="Compressed Activation Strength")
    plt.title("Activation Matrix (H)")
    plt.xlabel("Time [s]")
    plt.ylabel("Components")

    #Plotting the original target STFT magnitude V
    plt.subplot(2, 2, 3) #BOTTOMLEFT
    librosa.display.specshow(librosa.amplitude_to_db(V_compressed, ref=np.max), sr=fs, hop_length=int(time_res * fs), x_axis='time', y_axis='hz')
    plt.colorbar(format="%+2.0f dB")
    plt.title("Original Target Magnitude STFT (V)")
    plt.ylim(0, 2000)

    #Plotting the reconstructed magnitude STFT WH
    plt.subplot(2, 2, 4) #BOTTOMRIGHT
    librosa.display.specshow(librosa.amplitude_to_db(WH_compressed, ref=np.max), sr=fs, hop_length=int(time_res * fs), x_axis='time', y_axis='hz')
    plt.colorbar(format="%+2.0f dB")
    plt.title("Reconstructed Magnitude STFT (W * H)")
    plt.ylim(0, 2000)

    plt.tight_layout() 
    plt.show()

In [ ]:
# Test visualize_nmf
visualize_nmf(Y_target, W, H, fs=Fs, time_res=time_res, gamma=0.1)
visualize_nmf(Y_target, W, H, fs=Fs, time_res=time_res, gamma=0.5)
visualize_nmf(Y_target, W, H, fs=Fs, time_res=time_res, gamma=1)
visualize_nmf(Y_target, W, H, fs=Fs, time_res=time_res, gamma=2)


## Question 3

In [ ]:
# Replace the magnitude frames by complex valued frames
Y_transfer = Xs @ H

# Re-synthesize using ISTFT
print("Reconstructing waveform with ISTFT...")
y_istft = librosa.istft(Y_transfer, hop_length=H_size, win_length=N_length)

# Re-synthesize using Griffin-Lim algorithm
print("Reconstructing waveform with Griffin-Lim...")
y_gl = librosa.griffinlim(np.abs(Y_transfer), n_iter=32, hop_length=H_size, win_length=N_length)



In [ ]:
# Save result
istft_output_path = os.path.join(output_dir, f"{target.split('.')[0]}_{source.split('.')[0]}_istft.wav")
gl_output_path = os.path.join(output_dir, f"{target.split('.')[0]}_{source.split('.')[0]}_griffinlim.wav")

sf.write(istft_output_path, y_istft, Fs)
sf.write(gl_output_path, y_gl, Fs)

print(f"ISTFT result saved to: {istft_output_path}")
print(f"Griffin-Lim result saved to: {gl_output_path}")


In [ ]:
from scipy.fft import fft

# Plot the full signals
plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.plot(np.linspace(0, len(y_istft) / Fs, len(y_istft)), y_istft)
plt.title("y_istft (Full Signal)")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(np.linspace(0, len(y_gl) / Fs, len(y_gl)), y_gl)
plt.title("y_gl (Full Signal)")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.grid()

plt.tight_layout()
plt.show()


duration = 10  #Duration in seconds for analysis
num_samples = Fs * duration #Number of samples corresponding to the duration

#Truncating y_istft and y_gl to the first 10 seconds
y_istft_truncated = y_istft[:num_samples] #istft
y_gl_truncated = y_gl[:num_samples] #gl

# Compute FFT of the truncated signals
fft_istft = fft(y_istft_truncated)
fft_gl = fft(y_gl_truncated)

# Compute the phase using np.angle
phase_istft = np.angle(fft_istft)
phase_gl = np.angle(fft_gl)

# Unwrap the phase
unwrapped_phase_istft = np.unwrap(phase_istft)
unwrapped_phase_gl = np.unwrap(phase_gl)

# Frequency axis
frequencies = np.linspace(0, Fs, len(fft_istft))

# Plot the phases
plt.figure(figsize=(12, 8))

# Original phase of ISTFT
plt.subplot(2, 2, 1)
plt.plot(frequencies[:len(phase_istft)//2], phase_istft[:len(phase_istft)//2])
plt.title("Original Phase (ISTFT)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Phase (radians)")

# Unwrapped phase of ISTFT
plt.subplot(2, 2, 2)
plt.plot(frequencies[:len(unwrapped_phase_istft)//2], unwrapped_phase_istft[:len(unwrapped_phase_istft)//2])
plt.title("Unwrapped Phase (ISTFT)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Phase (radians)")

# Original phase of Griffin-Lim
plt.subplot(2, 2, 3)
plt.plot(frequencies[:len(phase_gl)//2], phase_gl[:len(phase_gl)//2])
plt.title("Original Phase (Griffin-Lim)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Phase (radians)")

# Unwrapped phase of Griffin-Lim
plt.subplot(2, 2, 4)
plt.plot(frequencies[:len(unwrapped_phase_gl)//2], unwrapped_phase_gl[:len(unwrapped_phase_gl)//2])
plt.title("Unwrapped Phase (Griffin-Lim)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Phase (radians)")

plt.tight_layout()
plt.show()

# Compute the phase difference (unwrapped)
phase_difference = unwrapped_phase_istft - unwrapped_phase_gl

# Plot the phase difference
plt.figure(figsize=(10, 6))

plt.plot(frequencies[:len(phase_difference)//2], phase_difference[:len(phase_difference)//2])
plt.title("Phase Difference (ISTFT vs Griffin-Lim)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Phase Difference (radians)")
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
plt.grid()
plt.tight_layout()
plt.show()



In [ ]:
# Play target
print("Playing original target signal...")
ipd.display(ipd.Audio(target_wave, rate=Fs))

In [ ]:
# Play source
print("Playing original source signal...")
ipd.display(ipd.Audio(source_wave, rate=Fs))

In [ ]:
# Play new target - ISTFT
print("Playing ISTFT result...")
ipd.display(ipd.Audio(istft_output_path))

In [ ]:
# Play new target - GF
print("Playing Griffin-Lim result...")
ipd.display(ipd.Audio(gl_output_path))

## Question 4

In [ ]:
# Define the function timbre_transfer
def timbre_transfer(t, s, fs, hop_size=1024, win_length=4096, resynth='gf', plot=False):
    """Transfer the timbre from the source to the target

    Args:
        t: target waveform
        s: source waveform
        fs: sampling frequency
        hop_size: hop size used for the STFT computation and the re-synthesis
        win_length: length of the window used for the STFT computation and the re-synthesis
        resynth: method used for the audio re-synthesis. Methods available: 'gf' = Griffin-Lim, 'istft' = Inverse STFT
        plot: boolean enabling visualization of NMF matrices and target spectrogram 

    Returns:
        y: audio waveform resynthesized through the chosen method
    """
    # Step 1: Compute STFTs
    X_target = librosa.stft(t, n_fft=win_length, hop_length=hop_size)
    X_source = librosa.stft(s, n_fft=win_length, hop_length=hop_size)

    Y_target = np.abs(X_target)
    Y_source = np.abs(X_source)

    # Normalize the source magnitude for template initialization
    eps = 1e-8  # Small value to avoid division by zero
    W0 = Y_source / (np.sum(Y_source, axis=0, keepdims=True) + eps)

    # Normalize the source STFT for Xs
    Xs = X_source / (np.sum(Y_source, axis=0, keepdims=True) + eps)

    # Random initialization for activation matrix H
    H0 = np.random.rand(X_source.shape[1], X_target.shape[1])

    # Perform NMF on the target magnitude using the source templates
    print("Performing NMF...")
    W, H = nmf(
        Y_target,
        num_iter=50,
        init_W=W0,
        init_H=H0,
        fix_W=True,
        cont_polyphony=10,
        cont_length=7,
        cont_grid=5,
        cont_sparsen=(1, 7)
    )

    # Approximate the target magnitude using NMF results
    Y_transfer = Xs @ H

    # Reconstruct the time-domain waveform
    if resynth == 'gf':
        print("Reconstructing waveform with Griffin-Lim...")
        y = librosa.griffinlim(np.abs(Y_transfer), n_iter=32, hop_length=hop_size, win_length=win_length)
    elif resynth == 'istft':
        print("Reconstructing waveform with ISTFT...")
        y = librosa.istft(Y_transfer, hop_length=hop_size, win_length=win_length)
    else:
        raise ValueError("Invalid resynthesis method. Choose 'gf' or 'istft'.")

    # Visualization if plot is enabled
    if plot:
        visualize_nmf(Y_target, H, W, time_res=hop_size / fs, gamma=2)
    return y

## Question 5

In [ ]:
# Use the function timbre_tansfer for all the possible combinations of source and target
if __name__ == "__main__":
    Fs = 22050

    sources = ["audio/sources/Bees_Buzzing.mp3", "audio/sources/Wind_Blowing.mp3", "audio/sources/Chainsaw_Sawing.mp3"]
    targets = ["audio/targets/Jingle_Bells_Boogie.wav", "audio/targets/Have_Yourself.wav", "audio/targets/Blue_Christmas.wav", "audio/targets/White_Christmas.wav"]

    for source_path in tqdm(sources, desc="Processing Sources"):
        source_wave, _ = librosa.load(source_path, sr=Fs)
        for target_path in tqdm(targets, desc="Processing Targets", leave=False):
            target_wave, _ = librosa.load(target_path, sr=Fs)

            # Perform timbre transfer
            transferred_wave = timbre_transfer(target_wave, source_wave, fs=Fs, resynth='gf', plot=False)

            # Save the result
            source_name = os.path.basename(source_path).split('.')[0]
            target_name = os.path.basename(target_path).split('.')[0]
            output_file = f"audio/results/{target_name}_{source_name}_transferred.wav"
            sf.write(output_file, transferred_wave, Fs)
            print(f"Saved: {output_file}")



In [ ]:
#N_length and Hop size tuning

def compute_frobenius_errors(sources, targets, fs=22050, hop_sizes=[512, 1024, 2048], n_lengths=[2048, 4096, 8192], num_iter=50):
    """
    Compute the Frobenius error for each combination of hop_size and N_length for all source-target pairs.

    Args:
        sources: List of source file paths.
        targets: List of target file paths.
        fs: Sampling frequency.
        hop_sizes: List of hop sizes to test.
        n_lengths: List of N lengths to test.
        num_iter: Number of iterations for NMF.

    Returns:
        results: A dictionary containing the best Frobenius error and corresponding parameters for each source-target pair.
    """
    results = []

    for source_path in tqdm(sources, desc="Processing Sources"):
        source_wave, _ = librosa.load(source_path, sr=fs)
        for target_path in tqdm(targets, desc="Processing Targets", leave=False):
            target_wave, _ = librosa.load(target_path, sr=fs)
            
            best_error = float('inf')
            best_params = None

            for hop_size in hop_sizes:
                for n_length in n_lengths:
                    try:
                        # STFT computation
                        X_source = librosa.stft(source_wave, n_fft=n_length, hop_length=hop_size)
                        X_target = librosa.stft(target_wave, n_fft=n_length, hop_length=hop_size)

                        Y_source = np.abs(X_source)
                        Y_target = np.abs(X_target)

                        # Normalize the source magnitude for template initialization
                        eps = 1e-8
                        W0 = Y_source / (np.sum(Y_source, axis=0, keepdims=True) + eps)

                        # Random initialization for activation matrix H
                        H0 = np.random.rand(X_source.shape[1], X_target.shape[1])

                        # Perform NMF on the target magnitude using the source templates
                        W, H = nmf(
                            Y_target,
                            num_iter=num_iter,
                            init_W=W0,
                            init_H=H0,
                            fix_W=True
                        )

                        # Compute Frobenius error
                        Y_approx = np.dot(W, H)
                        error = np.linalg.norm(Y_target - Y_approx, ord='fro')

                        # Update the best parameters if this error is lower
                        if error < best_error:
                            best_error = error
                            best_params = {
                                "source": source_path,
                                "target": target_path,
                                "hop_size": hop_size,
                                "n_length": n_length,
                                "error": best_error
                            }
                    except Exception as e:
                        print(f"Error with hop_size={hop_size}, n_length={n_length}: {e}")

            if best_params:
                results.append(best_params)

    return results

# Example usage
if __name__ == "__main__":
    sources = ["audio/sources/Bees_Buzzing.mp3", "audio/sources/Wind_Blowing.mp3", "audio/sources/Chainsaw_Sawing.mp3"]
    targets = ["audio/targets/Jingle_Bells_Boogie.wav", "audio/targets/Have_Yourself.wav", "audio/targets/Blue_Christmas.wav", "audio/targets/White_Christmas.wav"]

    hop_sizes = [512, 1024, 2048]
    n_lengths = [2048, 4096, 8192]

    results = compute_frobenius_errors(sources, targets, fs=22050, hop_sizes=hop_sizes, n_lengths=n_lengths)

    # Print results
    for result in results:
        print(f"Source: {result['source']}, Target: {result['target']}, Hop Size: {result['hop_size']}, N Length: {result['n_length']}, Error: {result['error']:.4f}")


In [ ]:
def compute_frobenius_for_configs(source, target, fs=22050, n_length=2048, hop_size=2048, num_iter=50, nmf_configs=None):
    """
    Compute Frobenius error for different configurations of NMF parameters on a single source-target pair.

    Args:
        source: Path to the source file.
        target: Path to the target file.
        fs: Sampling frequency.
        n_length: N length to use for STFT.
        hop_size: Hop size to use for STFT.
        num_iter: Number of iterations for NMF.
        nmf_configs: List of dictionaries with different NMF parameter configurations.

    Returns:
        results: A list of dictionaries containing Frobenius errors and the corresponding parameters.
    """
    if nmf_configs is None:
        nmf_configs = [
            {"fix_W": True, "cont_polyphony": 5, "cont_length": 5, "cont_grid": 5, "cont_sparsen": (1, 3)},
            {"fix_W": True, "cont_polyphony": 10, "cont_length": 7, "cont_grid": 5, "cont_sparsen": (1, 7)},
            {"fix_W": False, "cont_polyphony": 15, "cont_length": 10, "cont_grid": 10, "cont_sparsen": (3, 9)}
        ]

    results = []

    # Load source and target waves
    source_wave, _ = librosa.load(source, sr=fs)
    target_wave, _ = librosa.load(target, sr=fs)

    # Compute STFTs
    X_source = librosa.stft(source_wave, n_fft=n_length, hop_length=hop_size)
    X_target = librosa.stft(target_wave, n_fft=n_length, hop_length=hop_size)

    Y_source = np.abs(X_source)
    Y_target = np.abs(X_target)

    # Normalize the source magnitude for template initialization
    eps = 1e-8
    W0 = Y_source / (np.sum(Y_source, axis=0, keepdims=True) + eps)

    # Normalize the source STFT for Xs
    Xs = X_source / (np.sum(Y_source, axis=0, keepdims=True) + eps)

    # Test each NMF configuration
    for config in nmf_configs:
        try:
            # Random initialization for activation matrix H
            H0 = np.random.rand(X_source.shape[1], X_target.shape[1])

            # Perform NMF
            W, H = nmf(
                Y_target,
                num_iter=num_iter,
                init_W=W0,
                init_H=H0,
                fix_W=config["fix_W"],
                cont_polyphony=config["cont_polyphony"],
                cont_length=config["cont_length"],
                cont_grid=config["cont_grid"],
                cont_sparsen=config["cont_sparsen"]
            )

            # Compute Frobenius error
            Y_approx = np.dot(W, H)
            error = np.linalg.norm(Y_target - Y_approx, ord='fro')

            # Store results
            results.append({
                "n_length": n_length,
                "hop_size": hop_size,
                "error": error,
                "nmf_params": config
            })
            
            # Visualize the NMF result
            visualize_nmf(Y_target, W, H, fs, time_res=hop_size / fs, gamma = 0.1)

        
        except Exception as e:
            print(f"Error with n_length={n_length}, hop_size={hop_size}, config={config}: {e}")

    return results

# Example usage
if __name__ == "__main__":
    source = "audio/sources/Bees_Buzzing.mp3"
    target = "audio/targets/Jingle_Bells_Boogie.wav"

    n_length = 2048
    hop_size = 2048

    nmf_configs = [
        {"fix_W": True, "cont_polyphony": 5, "cont_length": 5, "cont_grid": 5, "cont_sparsen": (1, 3)},
        {"fix_W": True, "cont_polyphony": 10, "cont_length": 7, "cont_grid": 5, "cont_sparsen": (1, 7)},
        {"fix_W": False, "cont_polyphony": 15, "cont_length": 10, "cont_grid": 10, "cont_sparsen": (3, 9)}
    ]

    results = compute_frobenius_for_configs(source, target, fs=22050, n_length=n_length, hop_size=hop_size, nmf_configs=nmf_configs)

    # Print results
    for result in results:
        print(f"N Length: {result['n_length']}, Hop Size: {result['hop_size']}, Error: {result['error']:.4f}, Params: {result['nmf_params']}")


In [ ]:
def compare_timbres(audio1, audio2, sr=22050, n_mfcc=13):
    """
    Compare the timbre of two audio signals and return a similarity score.
    
    Args:
        audio1: Path to the first audio file.
        audio2: Path to the second audio file.
        sr: Sampling rate for audio processing.
        n_mfcc: Number of MFCCs to compute.
    
    Returns:
        A single similarity score.
    """
    # Load audio files
    y1, _ = librosa.load(audio1, sr=sr)
    y2, _ = librosa.load(audio2, sr=sr)
    
    # Compute MFCCs
    mfcc1 = librosa.feature.mfcc(y=y1, sr=sr, n_mfcc=n_mfcc)
    mfcc2 = librosa.feature.mfcc(y=y2, sr=sr, n_mfcc=n_mfcc)
    
    # Align features using Dynamic Time Warping (DTW)
    distance_matrix, path = librosa.sequence.dtw(mfcc1, mfcc2, metric='cosine')
    
    # Return the cumulative cost at the last cell
    return distance_matrix[-1, -1]

# Example usage
similarity_score = compare_timbres("audio/results/Jingle_Bells_Boogie_Bees_Buzzing_istft.wav",
                                   "audio/results/Jingle_Bells_Boogie_Bees_Buzzing_istft.wav")
print(f"Timbre similarity score: {similarity_score}")

similarity_score_1 = compare_timbres("audio/targets/Jingle_Bells_Boogie.wav",
                                     "audio/results/Jingle_Bells_Boogie_Bees_Buzzing_istft.wav")
print(f"Timbre similarity score: {similarity_score_1}")
